### This notebook to load, inspect, and play with your model!  Enjoy!

__Please don't hesitate to reach out with questions, bug fixes, new findings - whatever!__

In [123]:
import os, codecs
import gensim
from gensim import corpora
from collections import defaultdict
from string import punctuation
import pandas as pd
import numpy as np
from gensim.models import ldamodel
import itertools
from itertools import islice
import random

In [ ]:
#you are here

In [179]:
pwd

u'/Users/rachelbrynsvold/dsi/capstone_dir/Capstone/outputs-git_ignored/10p_5000_50'

Copy + paste this ^ string into the 'header' assignment

#### This should be all the information needed for this notebook's relative path assignment!
Confirm the dist_string matches the one you entered for your dimensional reduction and model fitting   
Confirm the header for the outputs_dir   
Make sure the outputs_dir exists

In [180]:
header = '/Users/rachelbrynsvold/dsi/capstone_dir/Capstone/outputs-git_ignored' + '/'

dist_string = '10p_5000_50'

outputs_dir = header + dist_string + '/'
outputs_dir

'/Users/rachelbrynsvold/dsi/capstone_dir/Capstone/outputs-git_ignored/10p_5000_50/'

Again copy + paste the above next to the cd, to change to the directory of interest and review the contents.

In [185]:
cd /Users/rachelbrynsvold/dsi/capstone_dir/Capstone/outputs-git_ignored/10p_5000_50/

/Users/rachelbrynsvold/dsi/capstone_dir/Capstone/outputs-git_ignored/10p_5000_50


In [187]:
#check what's in your outputs directory

In [183]:
ls

10p_5000_50.dict                    10p_5000_50.model.state
10p_5000_50.model                   10p_5000_50.model.state.sstats.npy
10p_5000_50.model.expElogbeta.npy   10p_5000_50_lst.txt
10p_5000_50.model.id2word


In [184]:
dict_ext = '.dict'
corp_ext = '_lst.txt'
model_ext = '.model'

dict_file_path = outputs_dir + dist_string + dict_ext
corp_file_path = outputs_dir + dist_string + corp_ext
model_file_path = outputs_dir + dist_string + model_ext
dict_file_path, corp_file_path, model_file_path
#make sure these look right

('/Users/rachelbrynsvold/dsi/capstone_dir/Capstone/outputs-git_ignored/10p_5000_50/10p_5000_50.dict',
 '/Users/rachelbrynsvold/dsi/capstone_dir/Capstone/outputs-git_ignored/10p_5000_50/10p_5000_50_lst.txt',
 '/Users/rachelbrynsvold/dsi/capstone_dir/Capstone/outputs-git_ignored/10p_5000_50/10p_5000_50.model')

**Load dictionary**

In [105]:
d = corpora.dictionary.Dictionary.load(dict_file_path)
print d, type(d)

Dictionary(2218001 unique tokens: [u'', u'thus--to', u'bison-tongue', u'with,[48', u'wire-jacket']...) <class 'gensim.corpora.dictionary.Dictionary'>


**Load/Inspect Streamed Corpus**

Some class objects for streaming the corpus, in both token and vector forms

In [67]:
class CorpStreamer(object):
    
    def __init__(self, dictionary, corp_lst_fp):
        self.dictionary = dictionary
        self.corp_lst_fp = corp_lst_fp
        
    def __iter__(self):
        with codecs.open(self.corp_lst_fp, 'r', encoding='utf_8') as f:
            for line in f:
                yield line.strip('/n').split(",")
                
class VecCorpStreamer(CorpStreamer):
    
    def __init__(self, dictionary, corp_lst_fp):
        self.dictionary = dictionary
        self.corp_lst_fp = corp_lst_fp
        self.corp_stream = CorpStreamer(self.dictionary, self.corp_lst_fp) 
    
    def __iter__(self):
        for line in self.corp_stream:
            yield self.dictionary.doc2bow(line)
    

In [107]:
#token or 'word' corpus stream - first entry printed
c = CorpStreamer(d, corp_file_path)
len(next(iter(c))), next(iter(c))

(3444,
 [u'retranscription',
  u'one',
  u'first',
  u'project',
  u'gutenberg',
  u'etexts',
  u'offically',
  u'dated',
  u'november',
  u'22',
  u'1973',
  u'officially',
  u're-released',
  u'november',
  u'22',
  u'1993',
  u'30th',
  u'anniversary',
  u'assassination',
  u'project',
  u'gutenberg',
  u'etext',
  u"kennedy's",
  u'inaugural',
  u'address',
  u"jfk's",
  u'inaugural',
  u'address',
  u'january',
  u'20',
  u'1961',
  u'12:11',
  u'est',
  u'observe',
  u'today',
  u'victory',
  u'party',
  u'celebration',
  u'freedom',
  u'',
  u'',
  u'symbolizing',
  u'end',
  u'well',
  u'beginning',
  u'',
  u'signifying',
  u'renewal',
  u'well',
  u'change',
  u'sworn',
  u'almighty',
  u'god',
  u'solemn',
  u'oath',
  u'forbears',
  u'prescribed',
  u'nearly',
  u'century',
  u'three-quarters',
  u'ago',
  u'world',
  u'different',
  u'man',
  u'holds',
  u'mortal',
  u'hands',
  u'power',
  u'abolish',
  u'forms',
  u'human',
  u'poverty',
  u'forms',
  u'human',
  u'life'

In [108]:
#vectorized corpus stream - first entry printed
vc = VecCorpStreamer(d, corp_file_path)
len(next(iter(vc))), next(iter(vc))

(1334,
 [(9, 4),
  (33, 1),
  (50, 13),
  (62, 1),
  (74, 1),
  (98, 1),
  (113, 16),
  (116, 1),
  (121, 2),
  (124, 4),
  (129, 1),
  (143, 1),
  (150, 3),
  (158, 3),
  (163, 2),
  (172, 1),
  (186, 4),
  (189, 4),
  (193, 5),
  (214, 3),
  (221, 2),
  (226, 3),
  (260, 2),
  (264, 2),
  (293, 3),
  (295, 24),
  (300, 2),
  (310, 5),
  (319, 19),
  (336, 3),
  (343, 1),
  (345, 1),
  (359, 1),
  (367, 1),
  (376, 1),
  (391, 4),
  (421, 1),
  (428, 3),
  (429, 1),
  (433, 3),
  (439, 4),
  (448, 3),
  (454, 2),
  (468, 5),
  (490, 6),
  (496, 5),
  (512, 1),
  (518, 3),
  (519, 1),
  (533, 1),
  (552, 4),
  (554, 4),
  (560, 2),
  (576, 1),
  (579, 8),
  (593, 1),
  (600, 1),
  (613, 199),
  (620, 1),
  (665, 3),
  (667, 2),
  (669, 11),
  (692, 1),
  (735, 2),
  (741, 3),
  (743, 3),
  (749, 1),
  (765, 3),
  (797, 1),
  (809, 1),
  (817, 4),
  (821, 1),
  (827, 9),
  (832, 1),
  (838, 3),
  (844, 4),
  (848, 2),
  (850, 2),
  (869, 1),
  (873, 1),
  (878, 1),
  (884, 1),
  (886, 5

**Load trained model**

In [109]:
lda = ldamodel.LdaModel.load(model_file_path)
print lda

LdaModel(num_terms=2218001, num_topics=50, decay=0.5, chunksize=2000)


In [110]:
num_topics = lda.num_topics
num_topics

50

**See the topics and their most significant terms**


In [126]:
all_topics_words = lda.print_topics(num_topics=num_topics)
all_topics_words

[(0,
  u'0.161*"" + 0.005*"one" + 0.003*"1" + 0.003*"new" + 0.003*"first" + 0.003*"would" + 0.002*"work" + 0.002*"may" + 0.002*"illustration" + 0.002*"mr"'),
 (1,
  u'0.025*"de" + 0.009*"madame" + 0.008*"n" + 0.006*"king" + 0.005*"paris" + 0.004*"la" + 0.004*"one" + 0.004*"monsieur" + 0.004*"v" + 0.003*"france"'),
 (2,
  u'0.011*"said" + 0.008*"would" + 0.007*"one" + 0.006*"could" + 0.005*"upon" + 0.005*"man" + 0.004*"know" + 0.004*"time" + 0.004*"well" + 0.004*"see"'),
 (3,
  u'0.005*"one" + 0.003*"great" + 0.003*"man" + 0.003*"vanslyperken" + 0.002*"made" + 0.002*"two" + 0.002*"king" + 0.002*"wi" + 0.002*"time" + 0.002*"like"'),
 (4,
  u'0.006*"1" + 0.005*"one" + 0.004*"2" + 0.004*"great" + 0.003*"1/2" + 0.003*"may" + 0.003*"also" + 0.003*"church" + 0.003*"two" + 0.003*"time"'),
 (5,
  u'0.005*"rachel" + 0.004*"janet" + 0.004*"one" + 0.004*"carve" + 0.003*"computer" + 0.003*"would" + 0.003*"louis" + 0.003*"people" + 0.002*"like" + 0.002*"well"'),
 (6,
  u'0.006*"one" + 0.006*"would" 

If desired, you can pick some (or all) of the topics above and assign them designations, being sure the number and designation match positions in their list.

In [116]:
topic_nums = [1, 13, 25, 43, 46, 49]
topic_designations = ['french', 'measurements', 'irish', 'christian', 'nautical', 'government']

topic_dict = {pr[0] : pr[1] for pr in zip(topic_nums, topic_designations)}

topic_dict

{1: 'french',
 13: 'measurements',
 22: 'irish',
 25: 'christian',
 43: 'nautical',
 46: 'government'}

**For a given document from corpus, see most relevant topics**

In [121]:
#We'll pull a random document out of our corpus by taking a slice of the stream\
#at a random number
#warning: potentially long run time (5-10mins)

In [138]:
r = random.randint(0, 5000)
rand_doc_words, rand_doc_vec = next(islice(vc, r, r+1)), next(islice(vc, r, r+1))

In [171]:
rand_doc_title = rand_doc[0]
print r
print rand_doc_title
print rand_doc

1403
pg
[u'pg', u'online', u'distributed', u'proofreading', u'team', u"transcriber's", u'note', u'footnotes', u'end', u'document', u'continental', u'monthly', u'devoted', u'literature', u'national', u'policy', u'vol', u'i.--may', u'1862.--no', u'v', u'', u'', u'', u'', u'', u'shall', u'first', u'blood', u'shed', u'revolutionary', u'struggle', u'boston', u'march', u'1770', u'next', u'lexington', u'june', u'1775', u'interval', u'filled', u'acts', u'coercion', u'oppression', u'one', u'side', u'complaints', u'remonstrances', u'thought', u'independence', u'entertained', u'people', u'even', u'time', u'affair', u'lexington', u'loyalty', u'mother', u'country', u'professed', u'even', u'clamorous', u'complaints', u'sincerely', u'great', u'majority', u'thought', u'redress', u'grievances', u'could', u'obtained', u'without', u'severance', u'great', u'britain', u'events', u'hurried', u'people', u'scarcely', u'spoken', u'beginning', u'struggle', u'soon', u'became', u'chief', u'object', u'present', u'

In [144]:
rand_doc_topics = lda.get_document_topics(rand_doc_vec, \
                        minimum_probability=.1)
rand_doc_topics

[(2, 0.11984712694112376),
 (23, 0.18788255859028566),
 (29, 0.41231835057745181)]

In [176]:
for n in rand_doc_topics:
    print lda.show_topic(n[0])
    print  ""

[(u'said', 0.010599240541684603), (u'would', 0.0081613418863971925), (u'one', 0.0074478754454131157), (u'could', 0.0055724298436422605), (u'upon', 0.0047889842859167066), (u'man', 0.0047298926013256762), (u'know', 0.0042079577968333884), (u'time', 0.0039343383083516219), (u'well', 0.0036884125251101223), (u'see', 0.003647379466688971)]

[(u'jurgis', 0.0051178804511877365), (u'one', 0.0050431497741997401), (u'ground', 0.0044526692929353559), (u'would', 0.0041408535714580396), (u'miles', 0.003540063365412642), (u'upon', 0.0033775107482124892), (u'fathoms', 0.0022268276783020137), (u'men', 0.0021698551915025183), (u'could', 0.0021578948813419149), (u'trueman', 0.0021337291535518358)]

[(u'one', 0.0070846522392706756), (u'would', 0.006427236945946508), (u'could', 0.0050912408344784702), (u'said', 0.004126144686815718), (u'man', 0.0035398459400203738), (u'little', 0.0035231984015357295), (u'upon', 0.0034894858624081211), (u'two', 0.003428371815531414), (u'us', 0.0033306485926752161), (u'tim

**For a given term in vocab, see what topics are most likely/relevant**

In [169]:
r1 = random.randint(0, len(d))
r1, d[r1]

(453227, u'lokuttara')

In [167]:
#if there's no result (empty list) try a lower number or a lower minimum_probability
lda.get_term_topics(500, minimum_probability=0.0001)

[]

**Human-readable topic terms**

In [170]:
for tup in lda.get_topic_terms(0):
    print d[tup[0]]


one
1
new
first
would
work
may
illustration
mr
